# Reducción de Ruido con Non-Local Means (NLM)

**NLM (Non-Local Means)** es un filtro para limpiar ruido que funciona de una manera muy diferente a los filtros tradicionales. La idea principal es que las imágenes reales tienen **patrones que se repiten en diferentes lugares**.

En lugar de limpiar cada píxel mirando solo a sus vecinos cercanos, NLM:

1. **Busca en toda la imagen**: Para limpiar un píxel, busca en toda la imagen (no solo alrededor) bloques pequeños que se parecen mucho al bloque donde está ese píxel.

2. **Promedia con peso**: Los bloques parecidos se usan para calcular un promedio, pero cada uno contribuye más o menos según **cuánto se parecen** al original.

3. **Preserva patrones**: Porque busca similitud global, los patrones y texturas se mantienen mucho mejor que con otros métodos.

La calidad del resultado depende de:

- **h (parámetro de filtrado)**: Controla la "fuerza" del filtrado. Valores altos limpian más pero pueden suavizar detalles. Valores bajos son más suaves con los detalles.

- **Tamaño del template**: El bloque pequeño que se busca. Más grande = busca similitudes menos exactas. Más pequeño = busca similitudes muy exactas.

- **Ventana de búsqueda**: Hasta dónde buscar bloques similares. Si es pequeña, es más rápido pero puede no encontrar todos los bloques parecidos.

### Cómo se Usa en VCF

En **VCF (Video Compression Framework)**, NLM se puede usar como:

1. **Preprocesamiento** (antes de codificar): Limpia la imagen antes de compresión
2. **Posprocesamiento** (después de decodificar): Limpia los artefactos que deja la compresión

```
Imagen Original → (Opcional: NLM) → Codificación → Almacenamiento → Decodificación → (NLM como Postfiltro) → Imagen Final
```

**Ventajas en VCF:**
- Muy bueno para preservar detalles mientras se limpia ruido
- Se puede usar antes de comprimir para mejorar la compresión
- Después de decodificar, recupera la calidad perdida por la compresión
- Flexible: puedes ajustar los parámetros según el contenido

## Integrantes del Proyecto

- **Isabel Pelaya Galindo Ibáñez**
- **Esther Ibáñez Mingorance**
- **José Luis López García**
- **Juan Rafael Madolell Usero**

In [ ]:
import subprocess
import sys

packages = ['matplotlib', 'scipy', 'bm3d']
for package in packages:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', package], check=False)

In [ ]:
%%writefile ../src/visualization.py
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

def show_images(original_path, encoded_path, decoded_path, title):
    """
    Función para visualizar y comparar las imágenes original, codificada y decodificada.
    
    Parámetros:
        original_path: ruta a la imagen original
        encoded_path: ruta a la imagen codificada (coeficientes)
        decoded_path: ruta a la imagen decodificada
        title: título para la visualización
    """
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(title, fontsize=16)
    
    # Mostrar imagen original
    if os.path.exists(original_path):
        img = cv2.cvtColor(cv2.imread(original_path), cv2.COLOR_BGR2RGB)
        axs[0].imshow(img)
        axs[0].set_title(f'Original\nTamaño: {os.path.getsize(original_path)/1024:.1f} KB')
    else:
        axs[0].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[0].axis('off')
    
    # Mostrar imagen codificada (Coeficientes)
    # Nota: Se muestra el archivo de coeficientes como imagen. Puede parecer ruido o bloques.
    if os.path.exists(encoded_path):
        try:
            # Intentar leer con OpenCV (maneja archivos TIF)
            img_enc = cv2.imread(encoded_path, cv2.IMREAD_UNCHANGED)
            if img_enc is not None:
                if len(img_enc.shape) == 3:
                     # Convertir BGR a RGB si es imagen de color
                     img_enc = cv2.cvtColor(img_enc, cv2.COLOR_BGR2RGB)
                # Mostrar los coeficientes sin normalización
                axs[1].imshow(img_enc, cmap='gray')
            else:
                 axs[1].text(0.5, 0.5, 'No se pudo leer TIF', ha='center')
        except Exception as e:
             axs[1].text(0.5, 0.5, f'Error: {e}', ha='center')
        
        # Calcular y mostrar tamaños de archivos
        weights_path = encoded_path.replace('.tif', '_weights.bin')
        w_size = os.path.getsize(weights_path) if os.path.exists(weights_path) else 0
        enc_size = os.path.getsize(encoded_path)
        total_size = enc_size + w_size
        axs[1].set_title(f'Coeficientes + Pesos\nTIF: {enc_size/1024:.1f} KB | Pesos: {w_size/1024:.1f} KB\nTotal: {total_size/1024:.1f} KB')
    else:
        axs[1].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[1].axis('off')
    
    # Mostrar imagen decodificada (reconstruida)
    if os.path.exists(decoded_path):
        img_dec = cv2.cvtColor(cv2.imread(decoded_path), cv2.COLOR_BGR2RGB)
        axs[2].imshow(img_dec)
        axs[2].set_title(f'Decodificada\nTamaño: {os.path.getsize(decoded_path)/1024:.1f} KB')
    else:
        axs[2].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()

def load_rgb(path):
    """Carga una imagen en formato RGB."""
    if not os.path.exists(path): return np.zeros((10,10,3), dtype=np.uint8)
    img = cv2.imread(path)
    if img is None: return np.zeros((10,10,3), dtype=np.uint8)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_results_bm3d(img_name, original_path, filtered_paths):
    """Muestra los resultados: Original + 3 filtradas (BM3D)."""
    # Configuración de niveles
    levels = ['Baja', 'Media', 'Alta']
    
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Imagen Original
    axes[0].imshow(load_rgb(original_path))
    axes[0].set_title(f"Original ({img_name})", fontsize=12, fontweight='bold')
    axes[0].axis('on')
    axes[0].set_xticks([]); axes[0].set_yticks([])
    
    # Resultados Filtrados (BM3D)
    for i in range(3):
        if i < len(filtered_paths):
            axes[i+1].imshow(load_rgb(filtered_paths[i]))
            axes[i+1].set_title(f"BM3D - {levels[i]}", fontsize=12)
            axes[i+1].axis('on')
            axes[i+1].set_xticks([]); axes[i+1].set_yticks([])
        
    plt.tight_layout()
    plt.suptitle(f"Resultados: {img_name}", fontsize=16, y=1.05)
    plt.show()

def show_results_nlm(img_name, original_path, filtered_paths):
    """Muestra los resultados: Original + 3 filtradas (NLM)."""
    # Configuración de niveles
    levels = ['Baja', 'Media', 'Alta']
    
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Imagen Original
    axes[0].imshow(load_rgb(original_path))
    axes[0].set_title(f"Original ({img_name})", fontsize=12, fontweight='bold')
    axes[0].axis('on')
    axes[0].set_xticks([]); axes[0].set_yticks([])
    
    # Resultados Filtrados (NLM)
    for i in range(3):
        if i < len(filtered_paths):
            axes[i+1].imshow(load_rgb(filtered_paths[i]))
            axes[i+1].set_title(f"NLM - {levels[i]}", fontsize=12)
            axes[i+1].axis('on')
            axes[i+1].set_xticks([]); axes[i+1].set_yticks([])
        
    plt.tight_layout()
    plt.suptitle(f"Resultados: {img_name}", fontsize=16, y=1.05)
    plt.show()

In [ ]:
%%writefile ../src/NLM.py
'''Eliminación de ruido en imágenes utilizando el filtro Non-Local Means (NLM). 
*** ¡Solo es efectivo durante la decodificación! ***'''

import numpy as np
import logging
import tempfile
import builtins
import os

# Create a valid temporary file path
temp_desc_path = os.path.join(tempfile.gettempdir(), "description.txt")

# Write the description to the valid temporary file
with open(temp_desc_path, 'w', encoding='utf-8') as f:
    f.write(__doc__)

# Monkeypatch open to redirect /tmp/description.txt to our valid temp file
# This is necessary because parser.py (which we cannot edit) hardcodes /tmp/description.txt
_original_open = builtins.open

def _redirect_open(file, *args, **kwargs):
    if file == "/tmp/description.txt":
        return _original_open(temp_desc_path, *args, **kwargs)
    return _original_open(file, *args, **kwargs)

builtins.open = _redirect_open

try:
    import parser
finally:
    # Restore original open
    builtins.open = _original_open
import main
import importlib
import cv2

default_h = 10
default_template_window_size = 7
default_search_window_size = 21

# Configuración del parser para la decodificación - Parámetros de NLM
# h: Determina la fuerza del filtro. Valores más altos eliminan más ruido pero pueden difuminar bordes.
# template_window_size: Tamaño del bloque que se usa para comparar similitudes.
# search_window_size: Tamaño del área donde se buscan bloques similares.
parser.parser_decode.add_argument("-h_nlm", "--h", type=float, help=f"Fuerza del filtro. Un h mayor elimina más ruido pero también detalles (por defecto: {default_h})", default=default_h)
parser.parser_decode.add_argument("-t", "--template_window_size", type=int, help=f"Tamaño del parche de plantilla (debe ser impar, por defecto: {default_template_window_size})", default=default_template_window_size)
parser.parser_decode.add_argument("-s", "--search_window_size", type=int, help=f"Tamaño del área de búsqueda (debe ser impar, por defecto: {default_search_window_size})", default=default_search_window_size)

import no_filter

args = parser.parser.parse_known_args()[0]

class CoDec(no_filter.CoDec):
    """
    Codec de NLM (Non-Local Means) para la eliminación de ruido.
    Hereda de no_filter.CoDec para manejar el flujo estándar de descompresión.
    """

    def __init__(self, args):
        logging.debug(f"trace args={args}")
        super().__init__(args)
        logging.debug(f"args = {self.args}")
        self.args = args

    def encode(self):
        """
        Método de codificación con corrección de argumentos.
        Necesario porque la clase base usa defaults incorrectos para este entorno.
        """
        img = self.encode_read(self.args.original)
        compressed_img = self.compress(img)
        output_size = self.encode_write(compressed_img, self.args.encoded)
        return output_size

    def decode(self):
        """
        Método de decodificación que aplica el filtro después de descomprimir los datos.
        """
        compressed_k = self.decode_read(self.args.encoded)
        k = self.decompress(compressed_k)
        logging.debug(f"k.shape={k.shape} k.dtype={k.dtype}")        
        y = self.filter(k)
        output_size = self.decode_write(y, self.args.decoded)
        return output_size
            
    def filter(self, img):
        """
        Aplica el algoritmo Non-Local Means de OpenCV.
        Detecta automáticamente si la imagen es en color o escala de grises para elegir la función adecuada.
        """
        logging.debug(f"trace y={img}")
        logging.info(f"NLM filter strength (h)={self.args.h}")
        logging.info(f"NLM template window size={self.args.template_window_size}")
        logging.info(f"NLM search window size={self.args.search_window_size}")
        
        if self.args.h < 0:
            logging.warning(f"Valor de h inválido ({self.args.h}). Debe ser no negativo. Ajustando a 0.")
            self.args.h = 0
            
        if self.args.template_window_size % 2 == 0 or self.args.template_window_size <= 0:
            logging.warning(f"Tamaño de ventana de plantilla inválido ({self.args.template_window_size}). Debe ser impar y positivo. Ajustando a {default_template_window_size}.")
            self.args.template_window_size = default_template_window_size

        if self.args.search_window_size % 2 == 0 or self.args.search_window_size <= 0:
            logging.warning(f"Tamaño de ventana de búsqueda inválido ({self.args.search_window_size}). Debe ser impar y positivo. Ajustando a {default_search_window_size}.")
            self.args.search_window_size = default_search_window_size

        try:
            # Verificar si la imagen es en color o escala de grises
            if len(img.shape) == 3:
                # Imagen en color - usar fastNlMeansDenoisingColored
                logging.info("Aplicando denoising NLM a imagen en color")
                return cv2.fastNlMeansDenoisingColored(
                    img, 
                    None, 
                    self.args.h, 
                    self.args.h,  # h para los componentes de color
                    self.args.template_window_size, 
                    self.args.search_window_size
                )
            else:
                # Imagen en escala de grises - usar fastNlMeansDenoising
                logging.info("Aplicando denoising NLM a imagen en escala de grises")
                return cv2.fastNlMeansDenoising(
                    img, 
                    None, 
                    self.args.h, 
                    self.args.template_window_size, 
                    self.args.search_window_size
                )
        except Exception as e:
            logging.error(f"Error durante el procesamiento NLM: {e}")
            logging.warning("Devolviendo imagen original debido al error.")
            return img

if __name__ == "__main__":
    main.main(parser.parser, logging, CoDec)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display, Image
import sys
sys.path.append('../src')
from visualization import show_results_nlm

In [ ]:
# Crear la carpeta /img si no existe
import os
img_dir = '../img'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    print(f"Carpeta '{img_dir}' creada exitosamente")
else:
    print(f"La carpeta '{img_dir}' ya existe")

## Parametros NLM
El NLM funciona buscando zonas parecidas en la imagen y promediándolas. Así quita el ruido sin borrar los bordes ni los detalles importantes.

Configuración:
- `-h` o `--h`: **Intensidad (fuerza)**. Determina cuánto ruido se elimina. Valores recomendados: 5 a 30.
- `-t` o `--template_window_size`: **Tamaño del parche**. Es el tamaño del bloque que se usa para comparar similitudes (debe ser impar, por defecto 7).
- `-s` o `--search_window_size`: **Radio de búsqueda**. Es la zona donde se buscan parches similares (debe ser impar, por defecto 21).

--- 
## Codificar imagen Pajaro.png

In [ ]:
# Codificar imagen original una sola vez
!python ../src/NLM.py encode -o /tmp/original.png -e ../img/Pajaro_encoded

## Comparación: Variando h (Fuerza del Filtrado)

El parámetro **h** controla cuánto "limpia" el filtro. Aquí lo variamos para que veas el efecto:

- **h = 5**: El filtrado es suave. Elimina poco ruido pero mantiene prácticamente todos los detalles. Si tu imagen no tiene mucho ruido, esta es buena opción.

- **h = 10**: Es el punto medio. Limpia un buen nivel de ruido sin ser demasiado agresivo. Funciona bien en la mayoría de los casos.

- **h = 20**: Aquí sí limpia bastante fuerte. El ruido se va bien pero los detalles finos pueden quedarse un poco blandos. Para imágenes con bastante ruido.

La diferencia entre valores altos y bajos es que con h bajo, el algoritmo es más "selectivo" a la hora de considerar bloques similares.

In [ ]:
# Decodificar con diferentes valores de h (fuerza del filtro)
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_h5.png --h 5
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_h10.png --h 10
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_h20.png --h 20
show_results_nlm('Pajaro.png (h=Fuerza)', '/tmp/original.png', ['../img/pajaro_nlm_h5.png', '../img/pajaro_nlm_h10.png', '../img/pajaro_nlm_h20.png'])

## Comparación: Variando el Tamaño del Template

El **template** es el bloque pequeño que el algoritmo busca en toda la imagen. Cambiar su tamaño afecta:

- **Template = 5×5**: Bloques pequeños, muy específicos. Busca similitudes casi exactas. Más lento pero puede encontrar coincidencias muy precisas.

- **Template = 7×7**: Tamaño medio. Es un buen balance entre precisión y velocidad. Funcionamiento general bastante bueno.

- **Template = 9×9**: Bloques más grandes, menos exactos. Encuentra similitudes más "aproximadas" pero es más rápido. Algunos detalles pueden no coincidir tan bien.

Con templates más grandes, el algoritmo es menos exigente con la similitud exacta, así que funciona más rápido pero puede no capturar los detalles tan bien.

In [ ]:
# Decodificar con diferentes tamaños de ventana de plantilla
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_t5.png --h 10 -t 5
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_t7.png --h 10 -t 7
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_t9.png --h 10 -t 9
show_results_nlm('Pajaro.png (Template=Parche)', '/tmp/original.png', ['../img/pajaro_nlm_t5.png', '../img/pajaro_nlm_t7.png', '../img/pajaro_nlm_t9.png'])

## Comparación: Variando la Ventana de Búsqueda

La **ventana de búsqueda** es la zona de la imagen donde el algoritmo busca bloques similares. Hacerla más o menos grande cambia mucho:

- **Ventana = 13×13**: Zona de búsqueda pequeña, muy localizada. El algoritmo busca bloques parecidos solo cerca del píxel actual. Más rápido pero puede perder similitudes que hay en otras partes de la imagen.

- **Ventana = 21×21**: Tamaño medio. Busca más lejos, encuentra más coincidencias posibles. Buen balance entre velocidad y resultados.

- **Ventana = 31×31**: Zona muy grande, busca en toda la imagen prácticamente. Encuentra las mejores similitudes posibles pero es bastante más lento.

Esto demuestra por qué NLM se llama "Non-Local": busca similitudes en toda la imagen, no solo localmente. Una ventana grande permite aprovechar mejor esa característica.

In [ ]:
# Comparación: Tamaño de área de búsqueda (Search Window Size)
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_s13.png --h 10 -s 13
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_s21.png --h 10 -s 21
!python ../src/NLM.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_nlm_s31.png --h 10 -s 31
show_results_nlm('Pajaro.png (Search=Búsqueda)', '/tmp/original.png', ['../img/pajaro_nlm_s13.png', '../img/pajaro_nlm_s21.png', '../img/pajaro_nlm_s31.png'])